In [33]:
import mysql.connector as sql


In [34]:
connection = sql.connect(
    host="localhost",
    user="root",
    password="Ayoub1111",
    allow_local_infile=True
)

In [35]:
cursor = connection.cursor()

In [36]:
cursor.execute("CREATE DATABASE IF NOT EXISTS product_sales")

In [37]:
cursor.execute('use product_sales')

In [38]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS discount (
        Month VARCHAR(255) NOT NULL,
        Discount_Band VARCHAR(255) NOT NULL,
        Discount INT NOT NULL
);
"""

cursor.execute(create_table_query)

In [39]:
connection.commit()

In [40]:
file_path = '/home/hp/Sql-Sales-Analysis/data/discount_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE discount
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Month, Discount_Band, Discount)
    """)


DatabaseError: LOAD DATA LOCAL INFILE file request rejected due to restrictions on access.

In [10]:
connection.commit()

In [11]:
cursor.execute("SHOW TABLES LIKE 'discount'")

In [12]:
result = cursor.fetchone()

In [13]:
if result:
    print("table exists")
else:
    print("table not exists")

table exists


In [14]:
table_name = 'discount'

In [15]:
query = f"SELECT * FROM {table_name}"

In [16]:
cursor.execute(query)
 

In [17]:
result = cursor.fetchall()

In [18]:
if result:
    for i in result:
        print(i)
else:
    print("table not found")

('January', 'none', 0)
('January', 'low', 6)
('January', 'medium', 10)
('January', 'high', 5)
('February', 'none', 0)
('February', 'low', 6)
('February', 'medium', 11)
('February', 'high', 5)
('March', 'none', 0)
('March', 'low', 6)
('March', 'medium', 4)
('March', 'high', 15)
('April', 'none', 0)
('April', 'low', 3)
('April', 'medium', 8)
('April', 'high', 15)
('May', 'none', 0)
('May', 'low', 7)
('May', 'medium', 11)
('May', 'high', 9)
('June', 'none', 0)
('June', 'low', 6)
('June', 'medium', 4)
('June', 'high', 7)
('July', 'none', 0)
('July', 'low', 8)
('July', 'medium', 9)
('July', 'high', 20)
('August', 'none', 0)
('August', 'low', 8)
('August', 'medium', 8)
('August', 'high', 17)
('September', 'none', 0)
('September', 'low', 5)
('September', 'medium', 7)
('September', 'high', 12)
('October', 'none', 0)
('October', 'low', 9)
('October', 'medium', 14)
('October', 'high', 20)
('November', 'none', 0)
('November', 'low', 15)
('November', 'medium', 19)
('November', 'high', 24)
('Decemb

In [37]:
product_query = """
    CREATE TABLE IF NOT EXISTS Products (
        ProductID VARCHAR(255) NOT NULL,
        Product VARCHAR(255) NOT NULL,
        Category VARCHAR(255) NOT NULL,
        Cost_Price VARCHAR(255) NOT NULL,
        Sale_Price INT NOT NULL,
        Brand VARCHAR(255) NOT NULL,
        Description VARCHAR(255) NOT NULL
);
"""

In [39]:
cursor.execute(product_query)

In [40]:
connection.commit()

In [36]:
file_path = '/home/hp/Sql-Sales-Analysis/data/Product_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE discount
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Product ID, Product, Category, Cost Price, Sale_Price, Brand, Description)
    """)


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ID, Product, Category, Cost Price, Sale_Price, Brand, Description)' at line 8